In [1]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image

np.set_printoptions(suppress=True, precision=2)

In [2]:
from data_processing.Processor import load_detection_ds, Processor
from matplotlib import cm as mp_cm
import cv2
import tqdm

In [3]:
from data_processing import image, bboxes
from utils.draw_bboxes import draw_bboxes
from models.fcos import build
from data_processing.io import load_gmc

In [4]:
raw_ds = load_detection_ds("training/temp_dataset.yaml")

for sample in raw_ds:
    break

In [5]:
processor = Processor([
    image.LoadImage(),
    bboxes.LoadBboxes(n_bboxes=90),
    image.ResizeKeepRatio(300,400),
    image.Normalize(),
    bboxes.BuildFCOSTarget((300,400), [8])
], feature_keys=None)

processed_ds = raw_ds.map(processor)

In [6]:
model = build.build_model()

# Detection Report

In [7]:
model.load_weights('models/fcos/snapshots/3e7a78331570bc3307935976e806ea78/weights.hdf5')

In [8]:
from models.fcos.interpreter import Interpreter
from data_processing.nms import nms
from data_processing import io
from data_processing.BboxDetectionReport import BboxDetectionReport

In [9]:
interpreter = Interpreter('configs/interpreter.yaml')

In [ ]:
bs = 1
for sample in tqdm.notebook.tqdm(raw_ds.map(processor).batch(bs)):
    raw_pred = model(sample['img'])
    bboxes_pred = interpreter(raw_pred,  sample["src_img_bbox"].numpy(), sample["original_shape"].numpy())
    bboxes_pred = nms(bboxes_pred, 0.2)
    io.serialize(sample['img_path'].numpy(), bboxes_pred, 'data/images', 'data/logs/val_report')

In [16]:
report = BboxDetectionReport()
report.from_serialized_report('data/images_markup', 'data/logs/val_report', class_idx = 0)
report.run_standard_evaluation()

{'precision_at_.8recall.5IoU': 0,
 'precision_at_.9recall.5IoU': 0,
 'recall_at_.8precision.5IoU': 0,
 'recall_at_.9precision.5IoU': 0}

In [17]:
report.pfr(0.1)

0